In [ ]:
from sympy.physics.units import *
from sympy import *

# Rounding:
import decimal
from decimal import Decimal as DX
def iso_round(obj, pv, rounding=decimal.ROUND_HALF_EVEN):
    import sympy
    """
    Rounding acc. to DIN EN ISO 80000-1:2013-08
    place value = Rundestellenwert
    """
    assert pv in set([
        # place value   #  round to:
        100,            #  3rd last digit before decimal
        10,             #  2nd last
        1,              #  last
        0.1,            #  1st digit after decimal
        0.01,           #  2nd
        0.001,          #  3rd
        0.0001,         #  4th
        0.00001,        #  5th
        0.000001,       #  6th
        0.0000001,      #  7th
        0.00000001,     #  8th
        0.000000001,    #  9th
        0.0000000001,   # 10th
        ])
    try:
        tmp = DX(str(float(obj)))
        obj = tmp.quantize(DX(str(pv)), rounding=rounding)
    except:
        for i in range(len(obj)):
            tmp = DX(str(float(obj[i])))
            obj[i] = tmp.quantize(DX(str(pv)), rounding=rounding)
    return obj

# LateX:
kwargs = {}
kwargs["mat_str"] = "bmatrix"
kwargs["mat_delim"] = ""
# kwargs["symbol_names"] = {FB: "F^{\mathsf B}", }

# Units:
(k, M, G ) = ( 10**3, 10**6, 10**9 )
(mm, cm, deg) = ( m/1000, m/100, pi/180)
Newton = kg*m/s**2
Pa     = Newton/m**2
MPa    = M*Pa
GPa    = G*Pa
kN     = k*Newton

# ---

# Solution with full matrix to find deformations and forces:

a, q, EI = var("a, q, EI")
# length of element:
l = a/4

# Klein:
sub_list_Klein=[
    (a, 2 *m),
    (q, 1 *Newton/m),
    (EI, 1 *Newton*m**2)
    ]

# ---

l2 = l*l
l3 = l*l*l

K = EI/l3
K *= Matrix(
[
[  4*l2 ,  -6*l ,  2*l2 ,   6*l ,  0    ,   0    ],
[ -6*l  ,  12   , -6*l  , -12   ,  0    ,   0    ],
[  2*l2 ,  -6*l ,  8*l2 ,   0   ,  2*l2 ,   6*l  ],
[  6*l  , -12   ,  0    ,  24   , -6*l  , -12    ],
[  0    ,   0   ,  2*l2 ,  -6*l ,  4*l2 ,   6*l  ],
[  0    ,   0   ,  6*l  , -12   ,  6*l  ,  12    ],
]
)
w1,p2,w2,p3 = var("w1,p2,w2,p3")
M1,F3 = var("M1,F3")
u = Matrix([0,w1,p2,w2,p3,0])
f = Matrix([M1,0,0,0,0,F3]) + q * Matrix([-l2/12, l/2, 0, l,l2/12, l/2 ])

unknowns = [w1,p2,w2,p3,M1,F3]

eq = Eq(K*u , f)
sol = solve(eq, unknowns)
pprint("\n\n---Solution 1 (FEM):")
w1,p2,w2,p3 = sol[w1],sol[p2],sol[w2],sol[p3]
pprint("\n w1, ψ2, w2, ψ3:")
for x in [w1,p2,w2,p3]:
    pprint(x)

# Solution with reduced matrix to find deformations only:
K = Matrix(
[
[    12   ,  -6*l  ,  -12 ,  0      ],
[  -6*l   ,  8*l2  ,    0 ,  2*l2   ],
[   -12   ,  0     ,   24 ,  -6*l   ],
[   0     ,  2*l2  ,  -6*l ,  4*l2  ],
]
)

K *= EI/l3

w1,p2,w2,p3 = var("w1,p2,w2,p3")
u = Matrix([w1,p2,w2,p3])
f = q * Matrix([l/2, 0, l, l2/12])
eq = Eq(K*u , f)
sol = solve(eq, [w1,p2,w2,p3])
pprint("\n\n--- Solution 2 (FEM):")
w1,p2,w2,p3 = sol[w1],sol[p2],sol[w2],sol[p3]
pprint("\n w1, ψ2, w2, ψ3:")
tmp = w1
tmp = tmp.subs(sub_list_Klein)
tmp /= m
tmp = iso_round(tmp, 0.001)
pprint(tmp)

tmp = p2
tmp = tmp.subs(sub_list_Klein)
tmp = iso_round(tmp, 0.001)
pprint(tmp)

tmp = w2
tmp = tmp.subs(sub_list_Klein)
tmp /= m
tmp = iso_round(tmp, 0.001)
pprint(tmp)

tmp = p3
tmp = tmp.subs(sub_list_Klein)
tmp = iso_round(tmp, 0.001)
pprint(tmp)
